# Process the Load Data for the NTP Heat Wave Grid Stress Events


In [ ]:
# Start by importing the packages we need:
import os
import glob
import datetime
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Set the Directory Structure

In [ ]:
# Identify the data input and image output directory:
data_dir =  '/Users/burl878/Documents/Code/code_repos/ntp_heat_wave_loads/data/'
plot_dir =  '/Users/burl878/Documents/Code/code_repos/ntp_heat_wave_loads/plots/'


## Suppress Future Warnings


In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)


## Create a Function to Process the 2035 GridView Data Used in Scaling


In [ ]:
def process_gridview_data(data_dir: str):
    
    # Read in the raw data .csv file:
    gv_df = pd.read_csv((data_dir + 'wecc_load_2035.csv'))
    
    # Subset to just the annual total demand by BA:
    gv_df = gv_df[-3:-2]
       
    # Strip the unecessary bits from the column names:
    gv_df.columns = gv_df.columns.str.replace("_CEC", "")
    gv_df.columns = gv_df.columns.str.replace("_2030.dat", "")
    gv_df.columns = gv_df.columns.str.replace("Load_", "")
       
    # Delete the index and last column:
    del gv_df["Index"], gv_df["Unnamed: 38"]
    
    # Compute the total loads for CISO, IPCO, NEVP, and PACE:
    gv_df['CISO'] = (gv_df['CIPB'] + gv_df['CIPV'] + gv_df['CISC'] + gv_df['CISD'] + gv_df['VEA']).round(2)
    gv_df['IPCO'] = (gv_df['IPFE'] + gv_df['IPMV'] + gv_df['IPTV']).round(2)
    gv_df['PACE'] = (gv_df['PAID'] + gv_df['PAUT'] + gv_df['PAWY']).round(2)
    gv_df['NEVP_Sum'] = (gv_df['NEVP'] + gv_df['SPPC']).round(2)
           
    # Rename a few columns for consistency:
    gv_df.rename(columns={'CIPB': 'CISO_CIPB', 'CIPV': 'CISO_CIPV', 'CISC': 'CISO_CISC', 'CISD': 'CISO_CISD', 'VEA': 'CISO_VEA',
                          'IPFE': 'IPCO_IPFE', 'IPMV': 'IPCO_IPMV', 'IPTV': 'IPCO_IPTV',
                          'NEVP': 'NEVP_NEVP', 'SPPC': 'NEVP_SPPC',
                          'PAID': 'PACE_PAID', 'PAUT': 'PACE_PAUT', 'PAWY': 'PACE_PAWY'}, inplace=True) 
    gv_df.rename(columns={'NEVP_Sum': 'NEVP'}, inplace=True) 
    
    # Squeeze the dataframe:
    gv_df = gv_df.squeeze().to_frame()
        
    # Rename the columns:
    gv_df.reset_index(inplace=True)
    gv_df = gv_df.rename(columns = {'index':'BA'})
    gv_df.rename(columns={gv_df.columns[1]: "Total_Load_MWh" }, inplace = True)
       
    # Sort the dataframe alphabetically by BA name:
    gv_df = gv_df.sort_values('BA')
       
    # Return the output dataframe:
    return gv_df


## Create a Function to Aggregate the Raw TELL MLP Output into a Single Dataframe:


In [ ]:
def aggregate_mlp_output_files(data_dir: str, year_to_process: str):
    
    # Create a list of all of the MLP output files in the "mlp_input_dir" and aggregate the files in that list:
    list_of_files = sorted(glob.glob(os.path.join(data_dir, 'TELL_Data', year_to_process, '*_mlp_output.csv')))

    # Loop over the list of MLP output files:
    for file in range(len(list_of_files)):

        # Read in the .csv file and replace missing values with nan:
        mlp_data = pd.read_csv(list_of_files[file]).replace(-9999, np.nan)

        # Rename the "Load" variable:
        mlp_data.rename(columns={'Load': 'Hourly_Load_MWh'}, inplace=True)

        # Replacing missing or negative loads with NaN:
        mlp_data.loc[~(mlp_data['Hourly_Load_MWh'] > 0), 'Hourly_Load_MWh'] = np.nan

        # Aggregate the output into a new dataframe:
        if file == 0:
            tell_df = mlp_data
        else:
            tell_df = pd.concat([tell_df, mlp_data])
    
    # Return the output dataframe:
    return tell_df


## Create a Function to Scale the TELL Output Based on the GridView 2035 Values:


In [ ]:
def scale_tell_loads(data_dir: str, year_to_process: str):
    
    # Aggregate the TELL MLP files:
    tell_df = aggregate_mlp_output_files(data_dir = data_dir,
                                         year_to_process = year_to_process)
    
    # Read in the processed GridView file and rename a column for consistency:
    gv_df = process_gridview_data(data_dir = data_dir)
    gv_df.rename(columns={'Total_Load_MWh': 'GV_Total_Load_MWh'}, inplace=True) 
    
    # Merge the tell_df and gv_df dataframes based on common BA names:
    merged_df = tell_df.merge(gv_df, on=['BA'])
    
    # Sum the hourly TELL loads by BA into annual total loads:
    merged_df['TELL_Total_Load_MWh'] = merged_df.groupby('BA')['Hourly_Load_MWh'].transform('sum')
    
    # Compute the scaling factors that force the annual total loads to agree:
    merged_df['Scaling_Factor'] = merged_df['GV_Total_Load_MWh'] / merged_df['TELL_Total_Load_MWh']
    
    # Compute the scaled hourly loads:
    merged_df['Hourly_Load_MWh_Scaled'] = merged_df['Hourly_Load_MWh'] * merged_df['Scaling_Factor']
    
    # Compute the hours since the start of the year:
    merged_df['Hour'] = ((pd.to_datetime(merged_df['Time_UTC']) - datetime.datetime(int(year_to_process), 1, 1, 0, 0, 0)) / np.timedelta64(1, 'h') + 1).astype(int)
    
    # Only keep the columns that are needed:
    scaled_tell_df = merged_df[['Hour', 'BA', 'Hourly_Load_MWh_Scaled']].copy()
    
    # Drop the rows with missing values (i.e., there is not a corresponding GridView load):
    scaled_tell_df = scaled_tell_df.dropna(how = 'any')
    
    # Rename the load variable and round it to 5 decimals:
    scaled_tell_df.rename(columns={'Hourly_Load_MWh_Scaled': 'Load_MWh'}, inplace=True)
    scaled_tell_df['Load_MWh'] = scaled_tell_df['Load_MWh'].round(5)
    
    # Return the output dataframe:
    return scaled_tell_df


## Create a Function to Format the Output for Ingest to GridView:


In [ ]:
def format_scaled_tell_loads(data_dir: str, year_to_process: str):
    
    # Process the GridView file:
    gv_df = process_gridview_data(data_dir = data_dir)
    
    # Compute the load fractions for the subregions:
    CIPB_LF = (gv_df.loc[(gv_df['BA'] == 'CISO_CIPB')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'CISO')]['Total_Load_MWh'].values[0])
    CIPV_LF = (gv_df.loc[(gv_df['BA'] == 'CISO_CIPV')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'CISO')]['Total_Load_MWh'].values[0])
    CISC_LF = (gv_df.loc[(gv_df['BA'] == 'CISO_CISC')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'CISO')]['Total_Load_MWh'].values[0])
    CISD_LF = (gv_df.loc[(gv_df['BA'] == 'CISO_CISD')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'CISO')]['Total_Load_MWh'].values[0])
    VEA_LF  = (gv_df.loc[(gv_df['BA'] == 'CISO_VEA' )]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'CISO')]['Total_Load_MWh'].values[0])
    IPFE_LF = (gv_df.loc[(gv_df['BA'] == 'IPCO_IPFE')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'IPCO')]['Total_Load_MWh'].values[0])
    IPMV_LF = (gv_df.loc[(gv_df['BA'] == 'IPCO_IPMV')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'IPCO')]['Total_Load_MWh'].values[0])
    IPTV_LF = (gv_df.loc[(gv_df['BA'] == 'IPCO_IPTV')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'IPCO')]['Total_Load_MWh'].values[0])
    NEVP_LF = (gv_df.loc[(gv_df['BA'] == 'NEVP_NEVP')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'NEVP')]['Total_Load_MWh'].values[0])
    SPPC_LF = (gv_df.loc[(gv_df['BA'] == 'NEVP_SPPC')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'NEVP')]['Total_Load_MWh'].values[0])
    PAID_LF = (gv_df.loc[(gv_df['BA'] == 'PACE_PAID')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PACE')]['Total_Load_MWh'].values[0])
    PAUT_LF = (gv_df.loc[(gv_df['BA'] == 'PACE_PAUT')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PACE')]['Total_Load_MWh'].values[0])
    PAWY_LF = (gv_df.loc[(gv_df['BA'] == 'PACE_PAWY')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PACE')]['Total_Load_MWh'].values[0])
    
    # Aggregate the TELL MLP files:
    scaled_tell_df = scale_tell_loads(data_dir = data_dir, 
                                      year_to_process = year_to_process)
   
    # Reshape the dataframe and drop the indexes:
    load_df = scaled_tell_df.pivot(index = 'Hour', columns = 'BA', values = 'Load_MWh')
    load_df = load_df.reset_index(drop=False)
    
    # Add back in the text to the column headers:
    load_df = load_df.add_suffix('_2030.dat')
    load_df = load_df.add_prefix('Load_')
    
    # Rename the time variable:
    load_df.rename(columns={'Load_Hour_2030.dat': 'Index'}, inplace=True)
    
    # Compute the loads for the subregions:
    load_df['Load_CIPB_2030_CEC.dat'] = load_df['Load_CISO_2030.dat'] * CIPB_LF
    load_df['Load_CIPV_2030_CEC.dat'] = load_df['Load_CISO_2030.dat'] * CIPV_LF
    load_df['Load_CISC_2030_CEC.dat'] = load_df['Load_CISO_2030.dat'] * CISC_LF
    load_df['Load_CISD_2030_CEC.dat'] = load_df['Load_CISO_2030.dat'] * CISD_LF
    load_df['Load_VEA_2030.dat'] = load_df['Load_CISO_2030.dat'] * VEA_LF
    load_df['Load_IPFE_2030.dat'] = load_df['Load_IPCO_2030.dat'] * IPFE_LF
    load_df['Load_IPMV_2030.dat'] = load_df['Load_IPCO_2030.dat'] * IPMV_LF
    load_df['Load_IPTV_2030.dat'] = load_df['Load_IPCO_2030.dat'] * IPTV_LF
    load_df['Load_NEVP_Temp_2030.dat'] = load_df['Load_NEVP_2030.dat'] * NEVP_LF
    load_df['Load_SPPC_2030.dat'] = load_df['Load_NEVP_2030.dat'] * SPPC_LF
    load_df['Load_PAID_2030.dat'] = load_df['Load_PACE_2030.dat'] * PAID_LF
    load_df['Load_PAUT_2030.dat'] = load_df['Load_PACE_2030.dat'] * PAUT_LF
    load_df['Load_PAWY_2030.dat'] = load_df['Load_PACE_2030.dat'] * PAWY_LF
    
    # Drop the un-needed columns for BAs with subregions:
    del load_df['Load_NEVP_2030.dat'], load_df['Load_CISO_2030.dat'], load_df['Load_IPCO_2030.dat'], load_df['Load_PACE_2030.dat']
    
    # Clean up the NEVP naming:
    load_df.rename(columns={'Load_NEVP_Temp_2030.dat': 'Load_NEVP_2030.dat'}, inplace=True)
    
    # Create a target dataframe with the spare hours:
    target_df = pd.DataFrame({"Index": np.arange(1,8791,1)})
    
    # Merge load dataframe with the target dataframe:
    merged_df = target_df.merge(load_df, on=['Index'], how='left')
    
    # Compute the summary statistics:
    stats_df = merged_df.apply(['mean','sum','max','min'])
    
    # Fix the summary statistic labels:
    stats_df.iloc[0, 0] = 'AVG'
    stats_df.iloc[1, 0] = 'SUM'
    stats_df.iloc[2, 0] = 'MAX'
    stats_df.iloc[3, 0] = 'MIN'
      
    # Sort the data by column name and make the Index column appear first:
    merged_df.rename(columns={'Index': 'AA'}, inplace=True)
    merged_df = merged_df.sort_index(axis = 1)
    merged_df.rename(columns={'AA': 'Index'}, inplace=True)
    
    # Add in a blank row and fill it with the year placeholder:
    merged_df.loc[-0.5] = 0
    merged_df = merged_df.sort_index().reset_index(drop=True)
    merged_df.iloc[0, :] = '2030'
    merged_df.at[0, 'Index'] = 'Year'
        
    # Merge the hourly load data and statistics dataframes together:
    output_df = pd.concat([merged_df, stats_df], axis=0)
    
    # Replace NaNs with blank values:
    output_df.replace(np.nan, "", regex=True)
    
    # Set the output filename:
    if year_to_process == '2055':
       output_filename = 'TELL_Loads_2035_Based_on_2015_Weather_With_Climate_Change.csv'
    if year_to_process == '2058':
       output_filename = 'TELL_Loads_2035_Based_on_2018_Weather_With_Climate_Change.csv'
    
    # Write out the dataframe to a .csv file:
    output_df.to_csv((os.path.join(data_dir, output_filename)), sep=',', index=False)
    
    # Return the output dataframe:
    return output_df


## Call the Necessary Functions to Process the Data:


In [ ]:
output_df = format_scaled_tell_loads(data_dir = data_dir,
                                     year_to_process = '2058')

output_df


## Create a Function to Create Summary Plots of the Output


In [ ]:
def create_summary_plots(data_dir: str, year_to_process: str, image_output_dir: str, image_resolution: int, save_images=False):
    
    # Read in the raw data .csv file:
    gv_df = pd.read_csv((data_dir + 'wecc_load_2035.csv'))
    
    # Strip the unecessary bits from the column names:
    gv_df.columns = gv_df.columns.str.replace("_CEC", "")
    gv_df.columns = gv_df.columns.str.replace("_2030.dat", "")
    gv_df.columns = gv_df.columns.str.replace("Load_", "")
    
    # Delete the random column:
    del gv_df["Unnamed: 38"]
    
    # Subset to just the annual total demand by BA:
    gv_stats = gv_df[8791:8795]
    
    # Reshape the dataframe and rename the index column:
    gv_stats_df = pd.melt(gv_stats, id_vars='Index', var_name='BA', value_name='GV_Value')
    gv_stats_df.rename(columns={'Index': 'Statistic'}, inplace=True)
    
    # Drop the first row and last 4 rows:
    gv_df = gv_df.tail(-1).iloc[:-4, :]
    
    # Compute the hourly total load across BAs:
    gv_df['GV_Total_Load_MWh'] = gv_df.sum(axis=1)
    
    # Copy the total load to a new dataframe:
    gv_load_df = gv_df[['Index', 'GV_Total_Load_MWh']].copy().iloc[0:8760, :]
    gv_load_df.rename(columns={'Index': 'Hour'}, inplace=True)
    
    # Set the output filename:
    if year_to_process == '2055':
       output_filename = 'TELL_Loads_2035_Based_on_2015_Weather_With_Climate_Change.csv'
    if year_to_process == '2058':
       output_filename = 'TELL_Loads_2035_Based_on_2018_Weather_With_Climate_Change.csv'
    
    # Read in the raw TELL data .csv file:
    tell_df = pd.read_csv((data_dir + output_filename))
    
    # Strip the unecessary bits from the column names:
    tell_df.columns = tell_df.columns.str.replace("_CEC", "")
    tell_df.columns = tell_df.columns.str.replace("_2030.dat", "")
    tell_df.columns = tell_df.columns.str.replace("Load_", "")
    
    # Subset to just the annual total demand by BA:
    tell_stats = tell_df[8791:8795]
    
    # Reshape the dataframe and rename the index column:
    tell_stats_df = pd.melt(tell_stats, id_vars='Index', var_name='BA', value_name='TELL_Value')
    tell_stats_df.rename(columns={'Index': 'Statistic'}, inplace=True)
    
    # Merge the two statistics dataframes together:
    stats_df = pd.merge(gv_stats_df, tell_stats_df, on=['Statistic', 'BA'])
    
    # Drop the first row and last 4 rows:
    tell_df = tell_df.tail(-1).iloc[:-4, :]
    
    # Compute the hourly total load across BAs:
    tell_df['TELL_Total_Load_MWh'] = tell_df.sum(axis=1)
    
    # Copy the total load to a new dataframe:
    tell_load_df = tell_df[['Index', 'TELL_Total_Load_MWh']].copy().iloc[0:8760, :]
    tell_load_df.rename(columns={'Index': 'Hour'}, inplace=True)
    
    # Merge the two load dataframes together:
    load_df = pd.merge(gv_load_df, tell_load_df, on=['Hour'])
    
    # Compute the min and max loads:
    min_load = load_df[['GV_Total_Load_MWh','TELL_Total_Load_MWh']].min().min()
    max_load = load_df[['GV_Total_Load_MWh','TELL_Total_Load_MWh']].max().max()
    
    # Subset the statistics dataframe:
    avg_df = stats_df.loc[stats_df['Statistic'] == 'AVG']; avg_min = 0.95*avg_df[['GV_Value','TELL_Value']].min().min(); avg_max = 1.05*avg_df[['GV_Value','TELL_Value']].max().max()
    sum_df = stats_df.loc[stats_df['Statistic'] == 'SUM']; sum_min = 0.95*sum_df[['GV_Value','TELL_Value']].min().min(); sum_max = 1.05*sum_df[['GV_Value','TELL_Value']].max().max()
    min_df = stats_df.loc[stats_df['Statistic'] == 'MIN']; min_min = 0.95*min_df[['GV_Value','TELL_Value']].min().min(); min_max = 1.05*min_df[['GV_Value','TELL_Value']].max().max()
    max_df = stats_df.loc[stats_df['Statistic'] == 'MAX']; max_min = 0.95*max_df[['GV_Value','TELL_Value']].min().min(); max_max = 1.05*max_df[['GV_Value','TELL_Value']].max().max()
    
    # Set the output filename:
    if year_to_process == '2055':
       load_filename = 'Load_Time_Series_2015_Weather_With_Climate_Change.png'
       stats_filename = 'Load_Statistics_2015_Weather_With_Climate_Change.png'
       plot_title_a = 'TELL Total Loads Based on 2015 Weather With Climate Change'
    if year_to_process == '2058':
       load_filename = 'Load_Time_Series_2018_Weather_With_Climate_Change.png'
       stats_filename = 'Load_Statistics_2018_Weather_With_Climate_Change.png'
       plot_title_a = 'TELL Total Loads Based on 2018 Weather With Climate Change'
    
    # Make the total load time series plots:
    plt.figure(figsize=(24, 12))
    plt.rcParams['font.size'] = 16
    
    plt.subplot(211)
    plt.plot(load_df['Hour'], load_df['GV_Total_Load_MWh'], color='black', linestyle='-', label='Original GridView Loads', linewidth=0.5)
    plt.xlim([0, 8760]); plt.xticks([0, 8760],['0','8760']); plt.xlabel('')
    plt.ylim([min_load, max_load]); plt.ylabel('WECC Total Load [MWh]')
    plt.legend(loc='upper left', prop={'size': 12})
    plt.title('Original Total Loads for 2035')
        
    plt.subplot(212)
    plt.plot(load_df['Hour'], load_df['TELL_Total_Load_MWh'], color='black', linestyle='-', label='TELL Total Loads', linewidth=0.5)
    plt.xlim([0, 8760]); plt.xticks([0, 8760],['0','8760']); plt.xlabel('Hour of Year [--->]')
    plt.ylim([min_load, max_load]); plt.ylabel('WECC Total Load [MWh]') 
    plt.legend(loc='upper left', prop={'size': 12})
    plt.title(plot_title_a)
        
    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       plt.savefig(os.path.join(image_output_dir, load_filename), dpi=image_resolution, bbox_inches='tight', facecolor='white')
    
    
    
    # Make the statistics plots:
    plt.figure(figsize=(24, 12))
    plt.rcParams['font.size'] = 16
    
    plt.subplot(221)
    plt.plot([avg_min, avg_max], [avg_min, avg_max], color='k', linestyle='-', linewidth=1)
    plt.scatter(avg_df['GV_Value'], avg_df['TELL_Value'], s=50, c='blue')
    plt.xlim([avg_min, avg_max]); plt.ylim([avg_min, avg_max]);
    plt.xlabel('Original GridView Average Load [MWh]')
    plt.ylabel('TELL Average Load [MWh]')
    plt.title('Balancing Authority Annual Average Load')
        
    plt.subplot(222)
    plt.plot([sum_min, sum_max], [sum_min, sum_max], color='k', linestyle='-', linewidth=1)
    plt.scatter(sum_df['GV_Value'], sum_df['TELL_Value'], s=50, c='blue')
    plt.xlim([sum_min, sum_max]); plt.ylim([sum_min, sum_max]);
    plt.xlabel('Original GridView Annual Total Load [MWh]')
    plt.ylabel('TELL Annual Total Load [MWh]')
    plt.title('Balancing Authority Annual Total Load')
        
    plt.subplot(223)
    plt.plot([min_min, min_max], [min_min, min_max], color='k', linestyle='-', linewidth=1)
    plt.plot([min_min, min_max], [min_min, 0.9*min_max], color='k', linestyle='--', linewidth=1)
    plt.plot([min_min, min_max], [min_min, 1.1*min_max], color='k', linestyle='--', linewidth=1)
    plt.scatter(min_df['GV_Value'], min_df['TELL_Value'], s=50, c='blue')
    plt.xlim([min_min, min_max]); plt.ylim([min_min, min_max]);
    plt.xlabel('Original GridView Minimum Load [MWh]')
    plt.ylabel('TELL Minimum Load [MWh]')
    plt.title('Balancing Authority Annual Minimum Load')
    
    plt.subplot(224)
    plt.plot([max_min, max_max], [max_min, max_max], color='k', linestyle='-', linewidth=1)
    plt.plot([max_min, max_max], [max_min, 0.9*max_max], color='k', linestyle='--', linewidth=1)
    plt.plot([max_min, max_max], [max_min, 1.1*max_max], color='k', linestyle='--', linewidth=1)
    plt.scatter(max_df['GV_Value'], max_df['TELL_Value'], s=50, c='blue')
    plt.xlim([max_min, max_max]); plt.ylim([max_min, max_max]);
    plt.xlabel('Original GridView Maximum Load [MWh]')
    plt.ylabel('TELL Maximum Load [MWh]')
    plt.title('Balancing Authority Annual Maximum Load')
    
    plt.subplots_adjust(wspace=0.15, hspace=0.4)
    
    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       plt.savefig(os.path.join(image_output_dir, stats_filename), dpi=image_resolution, bbox_inches='tight', facecolor='white')


In [ ]:
create_summary_plots(data_dir = data_dir,
                     year_to_process = '2058',
                     image_output_dir = plot_dir, 
                     image_resolution = 300, 
                     save_images = True)
